In [ ]:
import pandas as pd
import os

# --- 1. CARGA DE DATOS ---
usuario_base = os.path.expanduser("~")
ruta_gold_output = os.path.join(
    usuario_base, "Documents", "A-DataStack", "01-Proyectos", 
    "01-Data_PipelinesFibex", "02_Data_Lake", "gold_data", "Maestro_Empleados_Gold.parquet"
)   
df = pd.read_parquet(ruta_gold_output)
Filtro1 = df['OficinaSistema'].str.contains("RATTAN")
Filtro2 = df['OficinaSae'].str.contains("RATTAN")
df = df[Filtro1 | Filtro2]
df = df.groupby('OficinaSistema')['Nombre_Completo'].count().reset_index()
df = df.rename(columns={'Nombre_Completo': 'Cantidad_Abonados'})
df = df.sort_values(by='Cantidad_Abonados', ascending=False)

display(df.head(100
                ))

Revision de Oficinas Propias

In [ ]:
import pandas as pd
import os

# --- 1. CARGA DE DATOS ---
usuario_base = os.path.expanduser("~")
ruta_gold_output = os.path.join(
    usuario_base, "Documents", "A-DataStack", "01-Proyectos", 
    "01-Data_PipelinesFibex", "02_Data_Lake", "gold_data", "Afluencia_Gold.parquet"
)

df = pd.read_parquet(ruta_gold_output)

# --- 2. EL FILTRO (Aquí está la magia) ---
# Le decimos: "Quédate solo con las filas donde Tipo_Sede es IGUAL a 'PROPIA'"
df = df[df['Tipo_Sede'] == 'PROPIA']

# --- 3. AGRUPACIÓN ---
# Agrupamos, contamos y usamos reset_index() para que quede formato tabla
df_resultado = df.groupby('Oficina')['N° Abonado'].count().reset_index()

# Opcional: Renombrar la columna para que no se llame "N° Abonado" (que confunde porque ahora es una cuenta)
df_resultado = df_resultado.rename(columns={'N° Abonado': 'Cantidad_Abonados'})
df_resultado = df_resultado.sort_values(by='Cantidad_Abonados', ascending=False)
display(df_resultado.head(40))

In [6]:
import pandas as pd
import os

usuario_base= os.path.expanduser("~")
ruta_gold_output = os.path.join(usuario_base, "Documents", "A-DataStack",
    "01-Proyectos", "01-Data_PipelinesFibex", "02_Data_Lake", 
    "gold_data", "Ventas_Estatus_Gold.parquet"
)
df = pd.read_parquet(ruta_gold_output)
display(df)

,N° Abonado,Documento,Estatus,Fecha,Vendedor,Costo,Grupo Afinidad,Nombre Franquicia,Ciudad,Hora,Tipo de afluencia,Oficina
0,A61776,12612549,ACTIVO,2025-07-02,YOLIANA CAROLINA PAEZ OFIC ARAGUA,5.0,HOGAR,FIBEX ARAGUA,PALO NEGRO,08:04 AM,Ventas,ARAGUA
1,A61783,4224514,ACTIVO,2025-07-02,YOLIANA CAROLINA PAEZ OFIC ARAGUA,5.0,HOGAR,FIBEX ARAGUA,PALO NEGRO,09:09 AM,Ventas,ARAGUA
2,A61791,30923483,ACTIVO,2025-07-02,FRANKELLY GABRIELA GARCIA OFIC ARAGUA,20.0,HOGAR,FIBEX ARAGUA,MARACAY,10:02 AM,Ventas,ARAGUA
3,A61800,5577594,CORTADO,2025-07-02,FRANKELLY GABRIELA GARCIA OFIC ARAGUA,5.0,HOGAR,FIBEX ARAGUA,EL LIMóN,11:04 AM,Ventas,ARAGUA
4,A61820,16551410,CORTADO,2025-07-02,FRANKELLY GABRIELA GARCIA OFIC ARAGUA,20.0,HOGAR,FIBEX ARAGUA,MARACAY,12:24 PM,Ventas,ARAGUA
...,...,...,...,...,...,...,...,...,...,...,...,...
9042,T12885,507389502,OBSTRUCCION,2025-10-31,LUISA ALEXANDRA HEREDIA TERAN OFIC TINAQUILLO,39.0,PYMES,FIBEX TINAQUILLO,TINAQUILLO,08:56 AM,Ventas,TINAQUILLO
9043,T12886,19723466,POR INSTALAR,2025-10-31,WILMER SOLORZANO OFIC TINAQUILLO,39.0,HOGAR,FIBEX TINAQUILLO,TINAQUILLO,04:04 PM,Ventas,TINAQUILLO
9044,V128404,18574542,ACTIVO,2025-10-31,MAYRA PACHECO OFIC SAMBIL VALENCIA,39.0,HOGAR,FIBEX VALENCIA,VALENCIA,11:09 AM,Ventas,SAMBIL VALENCIA
9045,V128414,9829500,ACTIVO,2025-10-31,YOLFRAN POLANCO OFIC METROPOLIS VALENCIA,39.0,HOGAR,FIBEX VALENCIA,VALENCIA,12:08 PM,Ventas,METROPOLIS VALENCIA


In [ ]:
import pandas as pd
import os

# --- 1. CARGA DE DATOS ---
usuario_base = os.path.expanduser("~")
ruta_gold_output = os.path.join(
    usuario_base, "Documents", "A-DataStack", "01-Proyectos", 
    "01-Data_PipelinesFibex", "02_Data_Lake", "gold_data", "Afluencia_Gold.parquet"
)
df = pd.read_parquet(ruta_gold_output)
display(df.head(100))


In [ ]:
import pandas as pd
import os

# --- CONFIGURACIÓN ---
usuario_base = os.path.expanduser("~")
ruta_gold = os.path.join(
    usuario_base, "Documents", "A-DataStack", "01-Proyectos", 
    "01-Data_PipelinesFibex", "02_Data_Lake", "gold_data", "Afluencia_Gold.parquet")

print(f"📂 Cargando datos de: {ruta_gold}...")
df = pd.read_parquet(ruta_gold)

# --- 1. FILTRO DE CALIDAD: SOLO PROPIAS ---
# Normalizamos por si acaso hay diferencias de mayúsculas
if 'Tipo_Sede' in df.columns:
    df['Tipo_Sede'] = df['Tipo_Sede'].astype(str).str.upper().str.strip()
    filtro_propias = df['Tipo_Sede'] == 'PROPIA'
    df_propias = df[filtro_propias].copy()
else:
    print("⚠️  No encontré la columna 'Tipo_Sede'. Mostrando todo lo que NO sea 'ALIADO / AGENTE'...")
    # Fallback por si la columna no existe (filtro inverso)
    filtro_propias = ~df['Oficina'].str.contains("ALIADO|AGENTE", case=False, na=False)
    df_propias = df[filtro_propias].copy()

# --- 2. GENERAR RESUMEN POR OFICINA ---
# Contamos transacciones y vendedores únicos
resumen_oficinas = df_propias.groupby('Oficina').agg(
    Vendedores_Unicos=('Vendedor', 'nunique'),
    Total_Transacciones=('Vendedor', 'count')
).reset_index().sort_values(by='Total_Transacciones', ascending=False)

print("\n" + "="*80)
print("📊 RESUMEN: CARGA DE TRABAJO POR OFICINA PROPIA")
print("="*80)
print(resumen_oficinas.to_markdown(index=False))

# --- 3. DETALLE: ¿QUIÉN ESTÁ EN CADA OFICINA? ---
# Agrupamos por Oficina y Vendedor para ver quién es quién
detalle = df_propias.groupby(['Oficina', 'Vendedor']).size().reset_index(name='Transacciones')
detalle = detalle.sort_values(by=['Oficina', 'Transacciones'], ascending=[True, False])

print("\n" + "="*80)
print("🕵️  AUDITORÍA DETALLADA: VENDEDORES POR SEDE PROPIA")
print("="*80)

# Configuración para ver todo sin cortes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Mostramos el detalle
# Si la lista es muy larga, esto te permitirá revisarla con scroll
display(detalle)

# --- 4. VALIDACIÓN DE DUPLICADOS (GENTE EN DOS OFICINAS AL MISMO TIEMPO) ---
# Buscamos si algún vendedor aparece en más de una oficina PROPIA
vendedores_viajeros = detalle.groupby('Vendedor')['Oficina'].nunique().reset_index(name='Cant_Oficinas')
viajeros = vendedores_viajeros[vendedores_viajeros['Cant_Oficinas'] > 1]

if not viajeros.empty:
    print(f"\n⚠️  ALERTA: Se encontraron {len(viajeros)} vendedores operando en MÚLTIPLES Oficinas Propias:")
    lista_viajeros = viajeros['Vendedor'].tolist()
    # Mostramos dónde están esos viajeros
    display(detalle[detalle['Vendedor'].isin(lista_viajeros)].sort_values(by='Vendedor'))
else:
    print("\n✅  Todo en orden: Cada vendedor está asignado a una única Oficina Propia.")

In [ ]:
import pandas as pd
import os
from config import PATHS
from rich.console import Console
from rich.table import Table

console = Console()
console.rule("[bold red]🕵️‍♂️ AUDITORÍA FORENSE: INTERCOM SAN CARLOS[/]")

# 1. CARGA DE DATOS
path_maestro = os.path.join(PATHS["gold"], "Maestro_Empleados_Gold.parquet")
path_silver = os.path.join(PATHS["silver"], "Afluencia_Consolidada_Silver.parquet")

df_maestro = pd.read_parquet(path_maestro)
df_silver = pd.read_parquet(path_silver)

# --- PASO 1: ¿QUIÉN ES EL CULPABLE EN EL MAESTRO? ---
# Buscamos cualquier empleado cuya oficina sea 'INTERCOM SAN CARLOS LAS VEGAS'
# (Ojo: Ajusta el string si en tu sistema se llama ligeramente distinto, aquí uso búsqueda flexible)

target_office_keyword = "SAN CARLOS LAS VEGAS"

# Filtramos el maestro
col_ofi = 'OficinaSistema' if 'OficinaSistema' in df_maestro.columns else 'Oficina'
sospechosos = df_maestro[df_maestro[col_ofi].astype(str).str.contains(target_office_keyword, case=False, na=False)]

console.print(f"\n[cyan]1. Empleados Oficiales en '{target_office_keyword}':[/]")
if not sospechosos.empty:
    cols_view = [c for c in ['Nombre_Completo', 'OficinaSae', col_ofi] if c in sospechosos.columns]
    
    table = Table(title="Lista de Sospechosos (Maestro)")
    for c in cols_view: table.add_column(c)
    
    for _, row in sospechosos[cols_view].iterrows():
        table.add_row(*[str(val) for val in row])
    console.print(table)
    
    # Análisis de riesgo
    cortos = sospechosos[sospechosos['Nombre_Completo'].str.len() < 10]
    if not cortos.empty:
        console.print(f"[bold red]⚠ ALERTA: Hay {len(cortos)} nombres muy cortos que pueden causar falsos positivos![/]")
else:
    console.print("[yellow]No se encontraron empleados con esa oficina en el Maestro.[/]")


# --- PASO 2: ¿QUÉ VENTAS ESTÁN CAYENDO AHÍ? ---
# Buscamos en Silver las ventas asignadas a esa oficina
# Nota: Aquí buscamos en la columna 'Oficina' (que ya es la técnica asignada)
ventas_afectadas = df_silver[df_silver['Oficina'].astype(str).str.contains(target_office_keyword, case=False, na=False)]

console.print(f"\n[cyan]2. Muestra de Ventas Asignadas (Silver):[/]")
if not ventas_afectadas.empty:
    # Agrupamos por el nombre original del vendedor para ver patrones
    top_errores = ventas_afectadas['Vendedor'].value_counts().head(20).reset_index()
    top_errores.columns = ['Vendedor_Original_Sucio', 'Cantidad_Asignada']
    
    table2 = Table(title=f"Top Vendedores que cayeron en {target_office_keyword}")
    table2.add_column("Vendedor Original (Facts)", style="yellow")
    table2.add_column("Cantidad", justify="right")
    
    for _, row in top_errores.iterrows():
        table2.add_row(str(row['Vendedor_Original_Sucio']), str(row['Cantidad_Asignada']))
    console.print(table2)
else:
    console.print("[green]No hay ventas asignadas a esta oficina en Silver.[/]")

In [ ]:
import duckdb
import pandas as pd
import os

# Lista de tus archivos
archivos_gold = [
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Maestro_Empleados_Gold.parquet",
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Recaudacion_Gold.parquet",
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Reclamos_App_Gold.parquet",
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Reclamos_Banco_Gold.parquet",
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Reclamos_General_Gold.parquet",
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Ventas_Estatus_Gold.parquet",
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Ventas_Listado_Gold.parquet",
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Actualizacion_Datos_Gold.parquet",
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Afluencia_Gold.parquet",
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Atencion_Cliente_Gold.parquet",
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\ComeBackHome_Gold.parquet",
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Dim_Cliente.parquet",
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Estadistica_Abonados_Historico.parquet",
    r"C:\Users\josperez\Documents\A-DataStack\01-Proyectos\01-Data_PipelinesFibex\02_Data_Lake\gold_data\Llamadas_Cobranza_Gold.parquet"
]

reporte = []

print("Iniciando auditoría de duplicados (Corrección de Sintaxis)...\n")

for archivo in archivos_gold:
    nombre_archivo = os.path.basename(archivo)
    
    if os.path.exists(archivo):
        try:
            # CORRECCIÓN: Usamos una subquery para contar los distintos.
            # Esto es más compatible y evita el error de "Binder Error".
            query = f"""
            SELECT 
                (SELECT count(*) FROM '{archivo}') as total_filas,
                (SELECT count(*) FROM (SELECT DISTINCT * FROM '{archivo}')) as filas_unicas
            """
            
            result = duckdb.query(query).fetchone()
            
            total_filas = result[0]
            filas_unicas = result[1]
            duplicados = total_filas - filas_unicas
            
            status = "✅ OK" if duplicados == 0 else "❌ DUPLICADOS"
            
            reporte.append({
                "Archivo": nombre_archivo,
                "Total Filas": total_filas,
                "Duplicados": duplicados,
                "Estado": status
            })
            
        except Exception as e:
            reporte.append({"Archivo": nombre_archivo, "Total Filas": 0, "Duplicados": 0, "Estado": f"Error: {str(e)}"})
    else:
        reporte.append({"Archivo": nombre_archivo, "Total Filas": 0, "Duplicados": 0, "Estado": "⚠️ Archivo no encontrado"})

# Crear DataFrame para visualizar
df_audit = pd.DataFrame(reporte)

# Mostrar resultados
print(df_audit.to_markdown(index=False))

# Alerta si hay problemas
problemas = df_audit[df_audit['Duplicados'] > 0]
if not problemas.empty:
    print("\nATENCIÓN: Se encontraron archivos con duplicados:")
    print(problemas)

In [ ]:
import pandas as pd
import os

# --- 1. CARGA DE DATOS ---
usuario_base = os.path.expanduser("~")
ruta_gold_output = os.path.join(
    usuario_base, "Documents", "A-DataStack", "01-Proyectos", 
    "01-Data_PipelinesFibex", "02_Data_Lake", "gold_data", "Afluencia_Gold.parquet")

print(f"📂 Cargando archivo: {ruta_gold_output}...")
df = pd.read_parquet(ruta_gold_output)

# --- 2. FILTRO DE INTERÉS ---
# Filtramos por los nombres clave (buscando sin importar mayúsculas/minúsculas)
nombres_clave = ['WILLIAMS', 'FLAVIO', 'CRUZ', 'LEAL']
filtro = df['Vendedor'].str.upper().str.contains('|'.join(nombres_clave), na=False)
df_audit = df[filtro].copy()

# --- 3. PREPARACIÓN DEL TIEMPO ---
# Asegurarnos de que existe una columna de fecha. 
# Asumo que se llama 'Fecha' o 'Fecha_Entrada' por ser Afluencia. 
# Si tu columna tiene otro nombre, ajusta esta línea:
columna_fecha = 'Fecha'  # <--- AJUSTAR SI ES NECESARIO (ej. 'Created_At', 'Fecha_Visita')

if columna_fecha in df_audit.columns:
    # Convertimos a datetime por seguridad
    df_audit[columna_fecha] = pd.to_datetime(df_audit[columna_fecha])
    # Creamos una columna "Mes-Año" para agrupar (Ej: 2025-12)
    df_audit['Periodo'] = df_audit[columna_fecha].dt.to_period('M')
else:
    print(f"⚠️ No encontré la columna '{columna_fecha}'. Mostraremos totales generales.")
    df_audit['Periodo'] = 'Total General'

# --- 4. ANÁLISIS CRUZADO (La prueba de fuego) ---
# Agrupamos por Vendedor (que ahora debería tener la oficina) y por Mes
resultado = df_audit.groupby(['Vendedor', 'Periodo']).size().reset_index(name='Total_Afluencia')

# Ordenamos para ver la cronología
resultado = resultado.sort_values(by=['Vendedor', 'Periodo'])

print("\n--- 🕵️ AUDITORÍA DE CAMBIO DE OFICINA ---")
print("Si la lógica funcionó, deberías ver al mismo vendedor con apellidos/oficinas distintas en meses distintos.\n")
display(resultado)

# Opcional: Ver nombres únicos encontrados para confirmar si hay duplicados o variantes
print("\n--- Variantes de Nombres Encontradas ---")
print(df_audit['Vendedor'].unique())

In [ ]:
import pandas as pd
import os
import numpy as np
from config import PATHS

# --- RUTAS ---
# Ajusta si tus rutas son diferentes
ruta_maestro = os.path.join(PATHS["gold"], "Maestro_Empleados_Gold.parquet")
ruta_dim = os.path.join(PATHS["silver"], "Dim Oficinas.csv")

print("🕵️ INICIANDO DIAGNÓSTICO DE CRUCE...")

# 1. CARGAR MAESTRO (La Verdad)
df_maestro = pd.read_parquet(ruta_maestro)

# Filtramos a Williams para ver cómo está guardado REALMENTE
williams = df_maestro[df_maestro['Nombre_Completo'].str.contains("WILLIAMS", na=False)]

print("\n--- 1. ASÍ ESTÁ WILLIAMS EN EL MAESTRO (Target) ---")
# Mostramos las columnas clave. Fíjate bien en 'Combinado' y 'OficinaSistema'
display(williams[['Nombre_Completo', 'OficinaSistema', 'Combinado', 'Fecha_Origen']])

# 2. SIMULAR LO QUE HACE AFLUENCIA (Source)
# Cargamos el diccionario
try:
    df_dim = pd.read_csv(ruta_dim)
except:
    df_dim = pd.read_csv(ruta_dim, encoding='latin-1')

# Normalizamos el diccionario igual que en el script Gold
df_dim['Input_Original'] = df_dim['Input_Original'].astype(str).str.upper().str.strip()
df_dim['Nombre_Normalizado_Final'] = df_dim['Nombre_Normalizado_Final'].astype(str).str.upper().str.strip()
df_dim['Largo'] = df_dim['Input_Original'].str.len()
df_dim.sort_values(by='Largo', ascending=False, inplace=True)
mapa_oficinas = dict(zip(df_dim['Input_Original'], df_dim['Nombre_Normalizado_Final']))

# Texto de prueba (Copiado de tu tabla de error)
texto_sucio = "WILLIAMS CRUZ OFIC EL PARRAL VALENCIA"

print(f"\n--- 2. ASÍ SE PROCESA EN AFLUENCIA (Source) ---")
print(f"Texto original: '{texto_sucio}'")

# Aplicamos la misma lógica de tu script Gold
texto = texto_sucio.upper().strip()
oficina_detectada = "SIN OFICINA"

for keyword, oficial in mapa_oficinas.items():
    if keyword in texto:
        print(f"   ✅ Keyword encontrada: '{keyword}' -> Mapea a: '{oficial}'")
        oficina_detectada = oficial
        texto = texto.replace(keyword, "")
        break

# Limpieza de nombre
basura = ["OFICINA", "OFIC", "OFI", "ASESOR", "VENDEDOR", "ATC", "VALENCIA", "PTO LA CRUZ", "-", "."]
for b in basura:
    texto = texto.replace(b, "")
nombre_limpio = " ".join(texto.split())

llave_generada = f"{nombre_limpio} {oficina_detectada}"

print(f"   👤 Nombre Limpio: '{nombre_limpio}'")
print(f"   🏢 Oficina Detectada: '{oficina_detectada}'")
print(f"   🔑 LLAVE FINAL: '{llave_generada}'")

print("\n--- 3. EL VEREDICTO ---")
# Verificamos si esa llave existe en el maestro de Williams
match = williams[williams['Combinado'] == llave_generada]

if not match.empty:
    print("✅ ¡HAY MATCH! (Esto debería funcionar)")
else:
    print("❌ ¡NO HAY MATCH!")
    print(f"   Afluencia busca: '{llave_generada}'")
    print(f"   Maestro tiene:   {williams['Combinado'].tolist()}")
    print("\n💡 SOLUCIÓN: Cambia en 'Dim Oficinas.csv' el valor de 'Nombre_Normalizado_Final' para que sea IDÉNTICO al del Maestro.")

In [36]:
import pandas as pd
import os

# --- 1. RUTAS DE DATOS ---
usuario_base = os.path.expanduser("~")
RUTA_BASE = os.path.join(
    usuario_base, "Documents", "A-DataStack", "01-Proyectos", "01-Data_PipelinesFibex", 
    "02_Data_Lake", "gold_data", "Actualizacion_Datos_Gold.parquet")

# --- 2. CARGA ---
df = pd.read_parquet(RUTA_BASE)

# --- 3. PROCESO ---
# Paso A: Crear un sub-dataframe solo con las filas donde 'Fecha' es Nula (NaN/NaT)
# isna() devuelve True si está vacío
df_vacios = df[df['Fecha'].isna()]

# Paso B: Agrupar por Origen y contar el tamaño del grupo
# Usamos .size() porque cuenta la fila entera. Si usas .count() te daría 0.
resumen_vacios = df_vacios.groupby('Origen').size().reset_index(name='total_sin_fecha')

# --- 4. RESULTADO ---
print("Conteo de registros que NO tienen fecha:")
print(resumen_vacios)

Conteo de registros que NO tienen fecha:
                                              Origen  total_sin_fecha
0  DATA - ACTUALIZACION GESTION CC 18-11-25 A 18-...              351


In [26]:
import pandas as pd
import os

# --- 1. RUTAS DE DATOS ---
usuario_base = os.path.expanduser("~")
RUTA_BASE = os.path.join(
    usuario_base, "Documents", "A-DataStack", "01-Proyectos", "01-Data_PipelinesFibex", 
    "02_Data_Lake", "gold_data", "Ventas_Estatus_Gold.parquet")

# --- 2. CARGA ---
df = pd.read_parquet(RUTA_BASE)
df = df[(df['Fecha'] == pd.Timestamp('2026-01-16'))]
display(df)

,N° Abonado,Documento,Estatus,Fecha,Vendedor,Costo,Grupo Afinidad,Nombre Franquicia,Ciudad,Hora,Tipo de afluencia,Oficina
5620,A74729,6423313,ACTIVO,2026-01-16,YOLIANA CAROLINA PAEZ OFIC ARAGUA,0.0,HOGAR,FIBEX ARAGUA,PALO NEGRO,09:28 AM,Ventas,ARAGUA
5621,A74741,26870788,ACTIVO,2026-01-16,FRANKELLY GABRIELA GARCIA OFIC ARAGUA,39.0,HOGAR,FIBEX ARAGUA,EL LIMóN,10:37 AM,Ventas,ARAGUA
5622,A74789,4545114,ACTIVO,2026-01-16,FRANKELLY GABRIELA GARCIA OFIC ARAGUA,0.0,HOGAR,FIBEX ARAGUA,EL LIMóN,02:43 PM,Ventas,ARAGUA
5623,A74806,14684358,ACTIVO,2026-01-16,YOLIANA CAROLINA PAEZ OFIC ARAGUA,0.0,HOGAR,FIBEX ARAGUA,LA VICTORIA,03:57 PM,Ventas,ARAGUA
5624,A74810,9678862,ACTIVO,2026-01-16,ANGELICA ANGULO OFIC ARAGUA,0.0,HOGAR,FIBEX ARAGUA,MARACAY,04:23 PM,Ventas,ARAGUA
...,...,...,...,...,...,...,...,...,...,...,...,...
6245,V134126,25829118,ACTIVO,2026-01-16,EMANUEL ALEJANDRO HIDALGO OFIC PASEO LAS INDUS...,39.0,HOGAR,FIBEX VALENCIA,VALENCIA,04:12 PM,Ventas,PASEO LAS INDUSTRIAS
6246,V134130,133325030,ACTIVO,2026-01-16,YULETZY FABELO OFIC PASEO LAS INDUSTRIAS,39.0,PYMES,FIBEX VALENCIA,VALENCIA,04:43 PM,Ventas,PASEO LAS INDUSTRIAS
6247,V134132,10227572,ACTIVO,2026-01-16,INES REYES OFIC TORRE FIBEX VIÑEDO,39.0,HOGAR,FIBEX VALENCIA,VALENCIA,04:55 PM,Ventas,TORRE FIBEX VIÑEDO
6248,V134136,28019558,ACTIVO,2026-01-16,ISRAEL CASTILLO OFIC SAMBIL VALENCIA,29.0,HOGAR,FIBEX VALENCIA,VALENCIA,08:27 PM,Ventas,SAMBIL VALENCIA


In [25]:
import pandas as pd
import os

# --- 1. RUTAS DE DATOS ---
usuario_base = os.path.expanduser("~")
RUTA_BASE = os.path.join(
    usuario_base, "Documents", "A-DataStack", "01-Proyectos", "01-Data_PipelinesFibex", 
    "02_Data_Lake", "gold_data", "Afluencia_Gold.parquet")

# --- 2. CARGA ---
df = pd.read_parquet(RUTA_BASE)
df = df[(df['Tipo de afluencia'] == 'Ventas') & (df['Fecha'] == pd.Timestamp('2026-01-16'))]
display(df)

,N° Abonado,Estatus,Fecha,Vendedor,Grupo Afinidad,Nombre Franquicia,Ciudad,Tipo de afluencia,Oficina,Clasificacion,Hora,Mes,Estado_Sede,Tipo_Sede
5620,A74729,ACTIVO,2026-01-16,YOLIANA CAROLINA PAEZ OFIC ARAGUA,HOGAR,FIBEX ARAGUA,PALO NEGRO,Ventas,ARAGUA,None,09:28 AM,enero,ARAGUA,ALIADO / AGENTE
5621,A74741,ACTIVO,2026-01-16,FRANKELLY GABRIELA GARCIA OFIC ARAGUA,HOGAR,FIBEX ARAGUA,EL LIMóN,Ventas,ARAGUA,None,10:37 AM,enero,ARAGUA,ALIADO / AGENTE
5622,A74789,ACTIVO,2026-01-16,FRANKELLY GABRIELA GARCIA OFIC ARAGUA,HOGAR,FIBEX ARAGUA,EL LIMóN,Ventas,ARAGUA,None,02:43 PM,enero,ARAGUA,ALIADO / AGENTE
5623,A74806,ACTIVO,2026-01-16,YOLIANA CAROLINA PAEZ OFIC ARAGUA,HOGAR,FIBEX ARAGUA,LA VICTORIA,Ventas,ARAGUA,None,03:57 PM,enero,ARAGUA,ALIADO / AGENTE
5624,A74810,ACTIVO,2026-01-16,ANGELICA ANGULO OFIC ARAGUA,HOGAR,FIBEX ARAGUA,MARACAY,Ventas,ARAGUA,None,04:23 PM,enero,ARAGUA,ALIADO / AGENTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6244,V134122,ACTIVO,2026-01-16,EMANUEL ALEJANDRO HIDALGO OFIC PASEO LAS INDUS...,HOGAR,FIBEX VALENCIA,VALENCIA,Ventas,PASEO LAS INDUSTRIAS,None,03:21 PM,enero,CARABOBO,PROPIA
6245,V134126,ACTIVO,2026-01-16,EMANUEL ALEJANDRO HIDALGO OFIC PASEO LAS INDUS...,HOGAR,FIBEX VALENCIA,VALENCIA,Ventas,PASEO LAS INDUSTRIAS,None,04:12 PM,enero,CARABOBO,PROPIA
6246,V134130,ACTIVO,2026-01-16,YULETZY FABELO OFIC PASEO LAS INDUSTRIAS,PYMES,FIBEX VALENCIA,VALENCIA,Ventas,PASEO LAS INDUSTRIAS,None,04:43 PM,enero,CARABOBO,PROPIA
6247,V134132,ACTIVO,2026-01-16,INES REYES OFIC TORRE FIBEX VIÑEDO,HOGAR,FIBEX VALENCIA,VALENCIA,Ventas,TORRE FIBEX VALENCIA,None,04:55 PM,enero,CARABOBO,PROPIA
